# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [77]:
# read into a data fram 

import pandas as pd

df = pd.read_csv('tic-tac-toe.csv')

In [78]:
# inspect the dataset

df.describe()
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [79]:
# Convert the categorical values to numeric in all columns.

# convert features to numeric
X_categorical_cols = ['TL','TM','TR','ML','MM','MR','BL','BM','BR']
mapping = {
    'x': 1,
    'o': -1,
    'b': 0
}
for col in X_categorical_cols:
    if df[col].dtype == 'object':
        df[col] = df[col].map(mapping)

# convert target to numeric
y = df['class'].astype(int)

df.dtypes


TL       int64
TM       int64
TR       int64
ML       int64
MM       int64
MR       int64
BL       int64
BM       int64
BR       int64
class     bool
dtype: object

In [80]:
# separate input and output
X = df.drop(columns='class')
y #declared in previous cell
print(X.shape)
print(y.shape)

(958, 9)
(958,)


In [81]:
# normalize the data
# no need to normalize the data with the type of mapping we did on the previous cell.

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [82]:
# 1. split the training and test data

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y # to keep similar proportion of wins in 'class'
)

In [83]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# import adam for custom learning rate
from tensorflow.keras.optimizers import Adam

# define function to call when we want to train the model
def neural_network():
    # 2. Create a sequential model
    model = Sequential()
    
    # 3. add layers to the model
    # Hidden layers
    model.add(Dense(32, activation='relu', input_shape=(9,)))
    model.add(Dense(32, activation='relu', input_shape=(9,)))
    model.add(Dense(16, activation='relu'))
    # Output layer (2 classes, softmax)
    model.add(Dense(2, activation='softmax'))
   
  # specify learning rate
    optimizer = Adam(learning_rate=0.001)

    # 4. Compile model
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']  
        )
    
    return model
   
   
    # modelo sugerido por chatgpt. 
    # usando una sola neurona de output (representa probabilidad de que X gane partida) 
    # y sigmoid en vez de softmax
    '''
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(input_dim,)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    '''


In [84]:
# 5. Fit the model

model = neural_network()


history = model.fit(
    X_train,
    y_train,
    epochs=40,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

Epoch 1/40


c:\Users\santi\devtools\anaconda3\envs\ironhack\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5049 - loss: 0.7066 - val_accuracy: 0.6688 - val_loss: 0.6627
Epoch 2/40
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6732 - loss: 0.6417 - val_accuracy: 0.6753 - val_loss: 0.6235
Epoch 3/40
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7124 - loss: 0.5979 - val_accuracy: 0.6948 - val_loss: 0.5945
Epoch 4/40
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7435 - loss: 0.5603 - val_accuracy: 0.7273 - val_loss: 0.5701
Epoch 5/40
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7712 - loss: 0.5308 - val_accuracy: 0.7403 - val_loss: 0.5486
Epoch 6/40
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7778 - loss: 0.5017 - val_accuracy: 0.7597 - val_loss: 0.5351
Epoch 7/40
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7810 - loss: 0.4747 - val_accuracy: 0.7597 - val_loss: 0.5191
Epoch 8/40
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7958 - loss: 0.4512 - val_accuracy: 0.7792 - val_loss: 0.5015
Epo

In [85]:
# 6. Evaluate the model with test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

print(f"Test loss: {test_loss:.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")

Test loss: 0.0795
Test accuracy: 0.9844


In [86]:
# 7. Save the model
model.save("tic-tac-toe.keras")

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [87]:
#load model
from tensorflow.keras.models import load_model
import numpy as np

model = load_model("tic-tac-toe.keras")

# gets random numbers between 0 and the largest index number in the dataset
samples_n = 15
random_i = np.random.choice(len(X_test), size=samples_n, replace=False)

# gets X and y for the sample
X_sample = X_test.iloc[random_i]
y_sample = y_test.iloc[random_i]

#runs those samples through the model. y_results gets the output (couples of probabilities for each  row to be own by X and not)
y_results = model.predict(X_sample)

# gets the maximum value in each couple of outputs
y_pred = np.argmax(y_results, axis=1)

# compares with actual results
results = pd.DataFrame({
    "Predicted": y_pred,
    "Actual": y_sample.values
})

# to remove the index and be able to make a comparison
y_sample_values = y_sample.values

# array of predictions (bools)
correct_preds = (y_pred == y_sample_values)
# sum of correct predictions
n_correct = correct_preds.sum()
# ration of correct predictions
correct_ratio = n_correct / samples_n


# print summary

print(f"""
Results summary:
----------------
Total predictions: {samples_n}
Correct predictions: {n_correct}
Accuracy on sample: {correct_ratio}
""")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step

Results summary:
----------------
Total predictions: 15
Correct predictions: 15
Accuracy on sample: 1.0



## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# Adding one more layer of 32 neurons with same configuration moved accuracy from .86 to .96 and the guesses in the second part from 80% to 100%
# test loss 0.1324
# going up to 40 epocs: 
# Test loss: 0.0491
# Test accuracy: 0.9896
# making a smaller learning rate didn't have a big impact once the model was in those numbers

**Which approach(es) did you find helpful to improve your model performance?**

In [89]:
# epochs number had a big impact in improving test loss. I somehow assicoaited adding a layer more with accuracy.